# ECS 171 Group Project - Group 10 - ???

## Clean/Wrangle Data

"Process the data the same way the paper author did"

In [1]:
import pandas as pd

#Contains 81 features extracted from 21263 superconductors along with the critical temperature in the 82nd column
train = pd.read_csv('./../superconduct/train.csv')

#Contains the chemical formula broken up for all the 21263 superconductors from the train.csv file.
#The last two columns have the critical temperature and chemical formula.
formula = pd.read_csv('./../superconduct/unique_m.csv')

print("Training Data")
print(train.shape)
print(train.head())

print("\nChemical formulas")
print(formula.shape)
print(formula.head())

Training Data
(21263, 82)
   number_of_elements  mean_atomic_mass  wtd_mean_atomic_mass  \
0                   4         88.944468             57.862692   
1                   5         92.729214             58.518416   
2                   4         88.944468             57.885242   
3                   4         88.944468             57.873967   
4                   4         88.944468             57.840143   

   gmean_atomic_mass  wtd_gmean_atomic_mass  entropy_atomic_mass  \
0          66.361592              36.116612             1.181795   
1          73.132787              36.396602             1.449309   
2          66.361592              36.122509             1.181795   
3          66.361592              36.119560             1.181795   
4          66.361592              36.110716             1.181795   

   wtd_entropy_atomic_mass  range_atomic_mass  wtd_range_atomic_mass  \
0                 1.062396          122.90607              31.794921   
1                 1.057755    

## Replicate original XGBoost model

## Optimize XGBoost model with tuned hyperparameters

## Train and test a Random Forest model

## Optimize Random Forest model with tuned hyperparameters

## Research Bayesian Neural Networks
## Train and test a Bayesian Neural Network

## Optimize Bayesian Neural Network with tuned hyperparameters

## Analyze all model results and performances

## Frontend?

In [2]:
def getNumeric(input_series):
    new_series = input_series.str.split(n=1)
    for i in range(0,len(input_series)):
        new_series[i] = new_series[i][0]
    return pd.to_numeric(new_series)

wolfram_data = pd.read_csv('./../superconduct/wolfram_element_data.csv')
fie_data = pd.read_csv('./../superconduct/fie.csv')

wolfram_data['atomic_mass'] = getNumeric(wolfram_data['atomic_mass'])
wolfram_data['atomic_radius'] = getNumeric(wolfram_data['atomic_radius'])
wolfram_data['Density'] = getNumeric(wolfram_data['Density'])
wolfram_data['ElectronAffinity'] = getNumeric(wolfram_data['ElectronAffinity'])
wolfram_data['FusionHeat'] = getNumeric(wolfram_data['FusionHeat'])
wolfram_data['ThermalConductivity'] = getNumeric(wolfram_data['ThermalConductivity'])
wolfram_data.head()

element_data = wolfram_data
element_data.insert(3, "fie", fie_data,True)
element_data = element_data.set_index('Abbreviation')
element_data = element_data.drop(['Atomic_Number'],axis=1)
element_data['ElectronAffinity'] += 1.5
print(element_data.head())

              atomic_mass     fie  atomic_radius    Density  ElectronAffinity  \
Abbreviation                                                                    
H                1.008000  1312.0           53.0     0.0899              74.3   
He               4.002602  2372.3           31.0     0.1785               1.5   
Li               6.940000   520.2          167.0   535.0000              61.1   
Be               9.012183   899.5          112.0  1848.0000               1.5   
B               10.810000   800.6           87.0  2460.0000              28.2   

              FusionHeat  ThermalConductivity  Valence  
Abbreviation                                            
H                  0.558               0.1805        1  
He                 0.020               0.1513        0  
Li                 3.000              85.0000        1  
Be                 7.950             190.0000        2  
B                 50.000              27.0000        3  


In [3]:
#!Parsing!
#Check if input string is valid
def checkValid(input_string):
    for i in range(0,len(value)):
        if(not ((value[i].isupper()) or (value[i].islower()) or (value[i].isdigit()) or (value[i] == '.'))):
            return -1

#Adds missing ones after abbreviations
#HO2 -> H1O2
def addOnes(input_string):
    tempString = ""
    for i in range(0,len(input_string)):
        tempString += input_string[i]
        if(input_string[i].isupper() or input_string[i].islower()):
            if(i+1 == len(input_string) or input_string[i+1].isupper()):
                tempString += '1'
    return tempString

#Adds spaces after tokens
#H2O1 -> H 2 O 1
def addSpaces(input_string):
    tempString = ""
    
    for i in range(0,len(input_string)-1):
        tempString += input_string[i]
        if(input_string[i].isupper() or input_string[i].islower()):
            if(input_string[i+1].isdigit()):
                tempString += ' '
        elif(input_string[i].isdigit()):
            if(input_string[i+1].isupper()):
                tempString += ' '
    tempString += input_string[len(input_string)-1]
    return tempString

#Parse string into two lists
#List of abbreviations and list of numbers
def parseInput(input_string):
    checkValid(input_string)
    parsedString = addOnes(input_string)
    parsedString = addSpaces(parsedString)
    parsedList = parsedString.split(' ')
    
    abbreviations = []
    numbers = []
    for i in range(0,len(parsedList)):
        if(i%2 == 0):
            abbreviations.append(parsedList[i])
        else:
            numbers.append(float(parsedList[i]))
    
    return abbreviations, numbers

value = input("Test: ")
parsed = parseInput(value)
print(parsed)
            

Test: H2O
(['H', 'O'], [2.0, 1.0])


In [4]:
from math import log

def getRatios(input_list):
    total = sum(input_list)
    ratios = []
    for i in range(0,len(input_list)):
        ratios.append(input_list[i]/total)

    return ratios

def getValues(input_list,new_dataframe,colname):
    newlist = []
    for i in range(0,len(input_list)):
        newlist.append(element_data.loc[new_dataframe['Abb'][i]][colname])
    return newlist

def getW(input_list):
    total = sum(input_list)
    wlist = []
    for i in range(0,len(input_list)):
        wlist.append(input_list[i]/total)
    return wlist
        
def getInt(p,w):
    total = 0
    intList = []
    for i in range(0,len(p)):
        total += p[i]*w[i]
    for i in range(0,len(p)):
        intList.append((p[i]*w[i])/total)
    return intList
    
def mean(input_list,n):
    total = sum(input_list)/n

    return total
    
def weightedMean(input_list,p):
    total = 0
    for i in range(0,len(input_list)):
        total += input_list[i]*p[i]
    
    return total
    
def geometricMean(input_list,n):
    total = 1
    for i in range(0,len(input_list)):
        total = total*input_list[i]
    total = pow(total,1/n)

    return total
    
def weightedGeometricMean(input_list,p):
    total = 1
    for i in range(0,len(input_list)):
        total = total*pow(input_list[i],p[i])
    
    return total
    
def entropy(wList):
    total = 0
    for i in range(0,len(wList)):
        print(wList[i])
        total -= (wList[i]*log(wList[i]+0.000001))
    
    return total
    
def weightedEntropy(input_list):
    total = 0
    for i in range(0,len(input_list)):
        total = total - (input_list[i]*log(input_list[i]+0.000001))
        
    return total
    
def getRange(input_list):
    maximum = max(input_list)
    minimum = min(input_list)
    
    return maximum - minimum
    
def getWeightedRange(input_list,p):
    tempList = []
    for i in range(0,len(input_list)):
        tempList.append(p[i]*input_list[i])
    maximum = max(tempList)
    #max_index = input_list.index(maximum)
    minimum = min(tempList)
    #min_index = input_list.index(minimum)
    
    return maximum - minimum
    
def stdDev(input_list,mean,n):
    total = 0
    for i in range(0,len(input_list)):
        total += pow((input_list[i]-mean),2)
    total = (1/n)*total
    total = pow(total,0.5)
    
    return total
    
def weightedStdDev(input_list,p,wtdMean):
    total = 0
    for i in range(0,len(input_list)):
        total += p[i]*pow((input_list[i]-wtdMean),2)
    total = pow(total,0.5)
    
    return total
    
def makeSample(data):
    new_dataframe = pd.DataFrame(data)
    new_dataframe = new_dataframe.T
    new_dataframe.columns = ['Abb','Num']
    ratios = getRatios(new_dataframe['Num'])
    numValues = new_dataframe.shape[0]
    
    sampleList = []
    sampleList.append(numValues)
    
    #len(element_data.columns)
    for i in range(0,len(element_data.columns)):
        valueList = getValues(new_dataframe['Abb'],new_dataframe,element_data.columns[i])
        wList = getW(valueList)
        intList = getInt(ratios,wList)
        
        mu = mean(valueList,numValues)
        v = weightedMean(valueList,ratios)
        
        sampleList.append(mu)
        sampleList.append(v)
        sampleList.append(geometricMean(valueList,numValues))
        sampleList.append(weightedGeometricMean(valueList,ratios))
        sampleList.append(entropy(wList))
        sampleList.append(weightedEntropy(intList))
        sampleList.append(getRange(valueList))
        sampleList.append(getWeightedRange(valueList,ratios))
        sampleList.append(stdDev(valueList,mu,numValues))
        sampleList.append(weightedStdDev(valueList,ratios,v))
        
    print(sampleList)
        
    return sampleList
    
Listy = makeSample(parsed)
print(len(Listy))

0.05926971247133533
0.9407302875286646
0.4996382192772002
0.5003617807227998
0.5247524752475248
0.4752475247524752
0.05918756995193889
0.940812430048061
0.3427121771217712
0.6572878228782287
0.7153846153846154
0.28461538461538466
0.8716438091558818
0.12835619084411823
0.3333333333333333
0.6666666666666666
[2, 8.5035, 6.005, 4.0158426263986, 2.533210624773533, 0.22495136450426279, 0.35048137066156815, 14.991, 4.6610000000000005, 7.4955, 7.066825171178356, 1312.95, 1312.6333333333332, 1312.9496563082685, 1312.633027854131, 0.6931449187913397, 0.6367348935678336, 1.900000000000091, 436.69999999999993, 0.9500000000000455, 0.8956685895030031, 50.5, 51.33333333333333, 50.438080851673966, 51.27798986638827, 0.6919193094983018, 0.6204428589129865, 5.0, 19.33333333333333, 2.5, 2.3570226039551585, 0.7594500000000001, 0.5362666666666667, 0.3584230740340248, 0.22603925682691642, 0.22472421620742297, 0.3501779540750363, 1.3391, 0.4164, 0.6695500000000001, 0.6312577937912706, 108.4, 97.0333333333333